<a href="https://colab.research.google.com/github/machiwao/CCADMACL_EXERCISES_COM222ML/blob/main/Exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 2: Use Gradient Boost for Regression

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e12/overview



In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

## Dataset
Train, test and sample submission file can be found in this link
https://www.kaggle.com/competitions/playground-series-s4e12/data

## 1. Load the Data

In [ ]:
# put your answer here
df = pd.read_csv('train.csv')
dt = pd.read_csv('test.csv')

df['is_train'] = 1
dt['is_train'] = 0

sf = pd.read_csv('sample_submission.csv')
combined_df = pd.concat([df, dt], axis=0)
combined_df = combined_df.reset_index(drop=True)

## 2. Perform Data preprocessing

In [ ]:
# put your answer here
# Median Imputation for Age, Annual Income, Health Score, Credit Score, Vehicle Age, Insurance Duration
combined_df['Age'].fillna(combined_df['Age'].median(), inplace=True)
combined_df['Annual Income'].fillna(combined_df['Annual Income'].median(), inplace=True)
combined_df['Health Score'].fillna(combined_df['Health Score'].median(), inplace=True)
combined_df['Credit Score'].fillna(combined_df['Credit Score'].median(), inplace=True)
combined_df['Vehicle Age'].fillna(combined_df['Vehicle Age'].median(), inplace=True)
combined_df['Insurance Duration'].fillna(combined_df['Insurance Duration'].median(), inplace=True)

<ipython-input-4-d09a0083c444>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Age'].fillna(combined_df['Age'].median(), inplace=True)
<ipython-input-4-d09a0083c444>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [ ]:
# Mode Imputation for Number of Dependents and Previous Claims
combined_df['Number of Dependents'].fillna(combined_df['Number of Dependents'].mode()[0], inplace=True)
combined_df['Previous Claims'].fillna(combined_df['Previous Claims'].mode()[0], inplace=True)

<ipython-input-5-bd9a7671b3f1>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Number of Dependents'].fillna(combined_df['Number of Dependents'].mode()[0], inplace=True)
<ipython-input-5-bd9a7671b3f1>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when 

In [ ]:
# Mode Imputation for Marital Status, Occupation, Customer Feedback
combined_df['Marital Status'].fillna(combined_df['Marital Status'].mode()[0], inplace=True)
combined_df['Occupation'].fillna(combined_df['Occupation'].mode()[0], inplace=True)
combined_df['Customer Feedback'].fillna(combined_df['Customer Feedback'].mode()[0], inplace=True)

<ipython-input-6-e348c3256ee4>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Marital Status'].fillna(combined_df['Marital Status'].mode()[0], inplace=True)
<ipython-input-6-e348c3256ee4>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[co

In [ ]:
'''Extract date features (e.g., year, month, day, day of the week) and handle
missing dates by filling them with a specific default (e.g., the median date).'''
combined_df['Policy Start Date'] = pd.to_datetime(combined_df['Policy Start Date'])
combined_df['Year'] = combined_df['Policy Start Date'].dt.year
combined_df['Month'] = combined_df['Policy Start Date'].dt.month
combined_df['Day'] = combined_df['Policy Start Date'].dt.day
combined_df['Day of Week'] = combined_df['Policy Start Date'].dt.dayofweek

combined_df.drop('Policy Start Date', axis=1, inplace=True)

## 3. Create a Pipeline

In [ ]:
# put your answer here
df = combined_df[combined_df['is_train'] == 1]
dt = combined_df[combined_df['is_train'] == 0]

In [ ]:
# put your answer here
df.drop(['id', 'is_train'], axis=1, inplace=True)
dt.drop(['is_train'], axis=1, inplace=True)

<ipython-input-9-13ab13fe7736>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['id', 'is_train'], axis=1, inplace=True)
<ipython-input-9-13ab13fe7736>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt.drop(['is_train'], axis=1, inplace=True)


In [ ]:
X = df.drop('Premium Amount', axis=1)
y = df['Premium Amount']

In [ ]:
categorical = X.select_dtypes(include=['object']).columns
numerical = X.select_dtypes(exclude=['object']).columns

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical),
        ('cat', OneHotEncoder(), categorical)
    ])

In [ ]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(**params))
])

## 4. Train the Model

In [ ]:
# put your answer here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:05:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "loss", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Year', 'Month', 'Day', 'Day of Week'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index(['Gender', 'Marital Status', 'Education Level', 'Occupation'...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              loss='squared_error', max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=4, max_leaves=None,
                              min_child_weight=None, min_samples_split=5,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=500,
                              n_jobs=None, ...))])

## 5. Evaluate the Model

In [ ]:
# put your answer here
y_pred = pipeline.predict(X_test)

y_pred = np.maximum(y_pred, 0)
# Check RMSLE
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print("RMSLE:", rmsle)

RMSLE: 1.1571301172411488


## Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [ ]:
id = sf.pop('id')
y_pred = pipeline.predict(dt.drop('id', axis=1))

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'Premium Amount': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
